In [1]:
import os
import glob
import pandas as pd
os.chdir("C:/Users/Thomas/BigData/data(2)")
print("hei")

hei


In [2]:
path = "C:/Users/Thomas/BigData/data(2)"

In [3]:
#Merge csv files into one
all_files = glob.glob(os.path.join(path, "sales_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "merged.csv")

In [4]:
#Drop two bogus columns
data = pd.read_csv("C:/Users/Thomas/BigData/data(2)/merged.csv")
data = data.drop(columns="x_t")
data = data.drop(columns="perf")
data.head()

,Unnamed: 0,Unnamed: 0.1,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,0,0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,3,3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,4,4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [5]:
#Create new column month from Order Date Column
data["Month"] = data["Order Date"].astype(str).str[0] + data["Order Date"].astype(str).str[1]
data.head()

,Unnamed: 0,Unnamed: 0.1,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,0,0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",04
1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,na
2,2,2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",04
3,3,3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04
4,4,4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04


In [6]:
#This one is worng on purpose, fixed in next step
data["Month"] = pd.to_numeric(data["Month"])

ValueError: Unable to parse string "na" at position 1

In [7]:
#Convert Month to numeric, removing all non-numeric rows
data = data[pd.to_numeric(data['Month'], errors='coerce').notnull()]

In [8]:
#Create new column sum
data["Quantity Ordered"] = pd.to_numeric(data["Quantity Ordered"])
data["Price Each"] = pd.to_numeric(data["Price Each"])
data["Sum"] = (data["Quantity Ordered"] * data["Price Each"])
data.head()

,Unnamed: 0,Unnamed: 0.1,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sum
0,0,0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",04,23.90
2,2,2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",04,99.99
3,3,3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04,600.00
4,4,4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04,11.99
5,5,5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",04,11.99


In [9]:
#Create new dataframe for sum grouped by month
data2 = data.groupby(['Month'])["Sum"].sum().reset_index()
data2 = data2.sort_values(by=['Sum'], ascending=False).reset_index(drop=True)
data2

,Month,Sum
0,12,4613443.34
1,10,3736726.88
2,04,3390670.24
3,11,3199603.20
4,05,3152606.75
5,03,2807100.38
6,07,2647775.76
7,06,2577802.26
8,08,2244467.88
9,02,2202022.42


In [10]:
#Write both dataframes to csv
data.to_csv("data1.csv",index=False)
data2.to_csv("data2.csv",index=False)